Licensed under the MIT License.

Copyright (c) 2021-2025. All rights reserved.

# Optuna for Deep Learning

* Using CNN
* Flowers102 Data

In [2]:
import optuna
import numpy as np
import timeit

import tensorflow as tf
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop

tf.keras.backend.clear_session()
print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
print("Num GPUs:", len(physical_devices))

2.6.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs: 1


In [3]:
X_train = np.load('../../crystal_ball/data_collector/structured_data/flowers_X_train_norm.npy')
X_test = np.load('../../crystal_ball/data_collector/structured_data/flowers_X_test_norm.npy')
y_train = np.load('../../crystal_ball/data_collector/structured_data/flowers_y_train.npy')
y_test = np.load('../../crystal_ball/data_collector/structured_data/flowers_y_test.npy')

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(7169, 300, 300, 3) (7169, 1)
(1020, 300, 300, 3) (1020, 1)


In [4]:
N_TRAIN_EXAMPLES = X_train.shape[0]
N_VALID_EXAMPLES = X_test.shape[0]
BATCHSIZE = 128 
CLASSES = 102
EPOCHS = 100

img_x = X_train.shape[1]
img_y = X_train.shape[2]
channel = X_train.shape[3]
input_shape = (img_x, img_y, channel)

In [5]:
def objective(trial):
    clear_session()  # clear clutter from previous tf.keras session graphs

    model = Sequential()
    model.add(
        Conv2D(
            filters=trial.suggest_categorical("filters", [64, 128]),
            kernel_size=trial.suggest_categorical("kernel_size", [3, 5]),
            strides=trial.suggest_categorical("strides", [1, 2, 3]),
            activation=trial.suggest_categorical("activation", ["relu", "sigmoid", "tanh"]),
            input_shape=input_shape,
        )
    )
    model.add(Flatten())
    model.add(Dense(CLASSES, activation="softmax"))

    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=RMSprop(learning_rate=learning_rate), metrics=["accuracy"]
    )
    
    model.fit(
        X_train,
        y_train,
        validation_data=(X_test, y_test),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        verbose='auto',
    )

    # evaluate the model accuracy on the validation set
    score = model.evaluate(X_test, y_test, verbose=0)
    return score[1]

In [6]:
start = timeit.default_timer()

with tf.device("/cpu:0"):  # the batch size and image input size are too large for my GPU, so switch to CPU
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
                
stop = timeit.default_timer()
print('Time: ', stop - start)

[I 2021-09-03 23:21:45,067] A new study created in memory with name: no-name-547d81e0-dce5-4224-8448-ca3175f3f2d7


Epoch 1/100
57/57 [==============================] - 201s 3s/step - loss: 776.5707 - accuracy: 0.0133 - val_loss: 788.7289 - val_accuracy: 0.0098
Epoch 2/100
57/57 [==============================] - 175s 3s/step - loss: 977.8134 - accuracy: 0.0099 - val_loss: 1007.2542 - val_accuracy: 0.0098
Epoch 3/100
57/57 [==============================] - 239s 4s/step - loss: 641.5590 - accuracy: 0.0123 - val_loss: 643.6069 - val_accuracy: 0.0098
Epoch 4/100
57/57 [==============================] - 240s 4s/step - loss: 658.1166 - accuracy: 0.0107 - val_loss: 724.0651 - val_accuracy: 0.0098
Epoch 5/100
57/57 [==============================] - 248s 4s/step - loss: 618.3788 - accuracy: 0.0141 - val_loss: 704.7053 - val_accuracy: 0.0098
Epoch 6/100
57/57 [==============================] - 250s 4s/step - loss: 586.8652 - accuracy: 0.0127 - val_loss: 568.6459 - val_accuracy: 0.0098
Epoch 7/100
57/57 [==============================] - 254s 4s/step - loss: 484.1299 - accuracy: 0.0134 - val_loss: 545.5018 

[I 2021-09-04 05:49:49,190] Trial 0 finished with value: 0.08921568840742111 and parameters: {'filters': 64, 'kernel_size': 3, 'strides': 2, 'activation': 'sigmoid', 'learning_rate': 0.00013752890772023206}. Best is trial 0 with value: 0.08921568840742111.


Number of finished trials: 1
Best trial:
  Value: 0.08921568840742111
  Params: 
    filters: 64
    kernel_size: 3
    strides: 2
    activation: sigmoid
    learning_rate: 0.00013752890772023206
Time:  23287.0285255
